# Juan Esteban Acosta López

In [16]:

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Parcial2JuanAcosta").getOrCreate()


In [31]:
spark

In [17]:
#1
#[17 Ptos] Teniendo en cuenta el archivo departuredelays.csv aplique las siguientes transformaciones y acciones:
rdd = spark.sparkContext.textFile("data/departuredelays.csv")

In [18]:
#[4 Ptos] Cree un RDD a partir de la carga del archivo departuredelays.csv. Divida cada línea por el carácter coma (,)
split_rdd = rdd.map(lambda line: line.split(','))

In [19]:
#[2 Ptos] Imprima los diez primeros registros

# Tomar los primeros diez registros
first_10_records = split_rdd.take(10)

# Imprimir los registros de manera más legible
for i, record in enumerate(first_10_records, 1):
    print(f"Registro {i}:")
    print(f"Date: {record[0]}")
    print(f"Delay: {record[1]}")
    print(f"Distance: {record[2]}")
    print(f"Origin: {record[3]}")
    print(f"Destination: {record[4]}")
    print()

Registro 1:
Date: date
Delay: delay
Distance: distance
Origin: origin
Destination: destination

Registro 2:
Date: 01011245
Delay: 6
Distance: 602
Origin: ABE
Destination: ATL

Registro 3:
Date: 01020600
Delay: -8
Distance: 369
Origin: ABE
Destination: DTW

Registro 4:
Date: 01021245
Delay: -2
Distance: 602
Origin: ABE
Destination: ATL

Registro 5:
Date: 01020605
Delay: -4
Distance: 602
Origin: ABE
Destination: ATL

Registro 6:
Date: 01031245
Delay: -4
Distance: 602
Origin: ABE
Destination: ATL

Registro 7:
Date: 01030605
Delay: 0
Distance: 602
Origin: ABE
Destination: ATL

Registro 8:
Date: 01041243
Delay: 10
Distance: 602
Origin: ABE
Destination: ATL

Registro 9:
Date: 01040605
Delay: 28
Distance: 602
Origin: ABE
Destination: ATL

Registro 10:
Date: 01051245
Delay: 88
Distance: 602
Origin: ABE
Destination: ATL



In [20]:
#[3 Ptos] Imprima los cinco primeros registros correspondientes a los campos date y distance.
# Selecciona los campos 'date' y 'distance' de los primeros cinco registros
selected_data = split_rdd.map(lambda x: (x[0], x[2])).take(5)

# Imprime los resultados
for record in selected_data:
    print(f"Date: {record[0]}, Distance: {record[1]}")

Date: date, Distance: distance
Date: 01011245, Distance: 602
Date: 01020600, Distance: 369
Date: 01021245, Distance: 602
Date: 01020605, Distance: 602


In [21]:
#[3 Ptos] Imprima los cinco primeros registros correspondientes a los campos origen y destination filtrados por el destino igual a DTW

filtered_data = split_rdd.filter(lambda x: x[4] == 'DTW').map(lambda x: (x[3], x[4]))

# Tomar los primeros cinco registros
first_five_records = filtered_data.take(5)

# Imprimir los resultados
for record in first_five_records:
    print(f"Origin: {record[0]}, Destination: {record[1]}")

Origin: ABE, Destination: DTW
Origin: ABE, Destination: DTW
Origin: ABE, Destination: DTW
Origin: ABE, Destination: DTW
Origin: ABE, Destination: DTW


In [22]:
#[5 Ptos] Obtenga el total de minutos de retraso de aquellos registros donde el origen sea igual a SEA y el destino sea igual a SFO

# Filtrar los registros donde el origen es igual a 'SEA' y el destino es igual a 'SFO'
filtered_data = split_rdd.filter(lambda x: x[3] == 'SEA' and x[4] == 'SFO')

# Calcular el total de minutos de retraso
total_delay_minutes = filtered_data.map(lambda x: int(x[1])).reduce(lambda x, y: x + y)

print(f"Total de minutos de retraso de SEA a SFO: {total_delay_minutes} minutos")

Total de minutos de retraso de SEA a SFO: 22293 minutos


In [23]:
#2

departures_df = spark.read.csv("data/departuredelays.csv", header=True)
departures_df.createOrReplaceTempView("departures")

airports_df = spark.read.option("delimiter", "\t").csv("data/airport-codes-na.txt", header=True, inferSchema=True)
airports_df.createOrReplaceTempView("airports")

In [24]:
#[10 Ptos] Teniendo en cuenta los archivos departuredelays.csv y airpot-codes-na.txt resolver la siguiente consulta: Listar los vuelos retrasados por estado y filtrados por el país USA. La lista debe estar ordenada por los vuelos retrasados en orden ascendente. Para la carga de los archivos tenga en cuenta lo siguiente: departuredelays.csv: Cree una vista temporal que incluya la cabecera airpot-codes-na.txt: Cree una vista temporal que incluya la cabecera, infiera el esquema y el separador de columnas sea \t

query = """
SELECT a.State, COUNT(*) AS DelayedFlights
FROM departures AS d
JOIN airports AS a ON d.origin = a.IATA
WHERE a.Country = 'USA' AND d.delay > 0
GROUP BY a.State
ORDER BY DelayedFlights ASC
LIMIT 20
"""

result = spark.sql(query)

result.show()

+-----+--------------+
|State|DelayedFlights|
+-----+--------------+
|   WV|           248|
|   VT|           257|
|   ME|           388|
|   WY|           565|
|   NH|           598|
|   ND|           765|
|   SD|           770|
|   KS|           808|
|   MT|           933|
|   RI|          1080|
|   MS|          1130|
|   ID|          1228|
|   AK|          1642|
|   IA|          1825|
|   KY|          1837|
|   NE|          1967|
|   CT|          2015|
|   AR|          2256|
|   SC|          2286|
|   NM|          2492|
+-----+--------------+



In [25]:
#3
#[23 Ptos] Con base en los datos de la tabla aplique las siguientes transformaciones y acciones:

from pyspark.sql.types import StructType, StructField, LongType, DoubleType, StringType

#[6 Ptos] Crear un DataFrame

# Define los datos y las columnas
data = [(1, 144.5, 5.9, 33, 'M'),
        (2, 167.2, 5.4, 45, 'M'),
        (3, 124.1, 5.2, 23, 'F'),
        (4, 144.5, 5.9, 33, 'M'),
        (5, 133.2, 5.7, 54, 'F'),
        (3, 124.1, 5.2, 23, 'F'),
        (5, 129.2, 5.3, 42, 'M')]

columns = ["id", "weight", "height", "age", "gender"]

# Define el esquema con los tipos de datos 
schema = StructType([
    StructField("id", LongType(), True),
    StructField("weight", DoubleType(), True),
    StructField("height", DoubleType(), True),
    StructField("age", LongType(), True),
    StructField("gender", StringType(), True)
])

# Crea un DataFrame a partir de los datos y el esquema corregido
df = spark.createDataFrame(data, schema=schema)

# Muestra el DataFrame
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [26]:
#[3 Ptos] Imprimir el esquema del DataFrame

# Imprimir el esquema del DataFrame
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)



In [27]:
#[3 Ptos] Contar la cantidad total de registros

# Contar la cantidad total de registros en el DataFrame
total_records = df.count()

print(f"La cantidad total de registros en el DataFrame es: {total_records}")

La cantidad total de registros en el DataFrame es: 7


In [28]:
#[3 Ptos] Contar la cantidad total de registros sin duplicados

# Eliminar registros duplicados y contar la cantidad total de registros únicos
unique_records_count = df.distinct().count()

print(f"La cantidad total de registros sin duplicados es: {unique_records_count}")

La cantidad total de registros sin duplicados es: 6


In [29]:
#[4 Ptos] Imprimir aquellos registros donde el id sea igual a 3

# Filtrar y mostrar registros donde "id" sea igual a 3
filtered_df = df.filter(df["id"] == 3)
filtered_df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  3| 124.1|   5.2| 23|     F|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [30]:
#[4 Ptos] Seleccionar los campos id y gender y filtrar por age mayor o igual a 30

# Seleccionar los campos "id" y "gender" y filtrar por "age" mayor o igual a 30
selected_df = df.select("id", "gender").filter(df["age"] >= 30)
selected_df.show()

+---+------+
| id|gender|
+---+------+
|  1|     M|
|  2|     M|
|  4|     M|
|  5|     F|
|  5|     M|
+---+------+

